In [ ]:
#@title ðŸ“¦ **STEP 1: Install & Setup** (Run this first)

print("ðŸ“¦ Installing required packages...\n")

!apt-get update -qq > /dev/null 2>&1
!apt-get install -y mkvtoolnix aria2 mediainfo -qq > /dev/null 2>&1
!pip install -q gdown

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from google.colab import files
import os
import shutil
import gdown
import subprocess
import json
from pathlib import Path

WORK_DIR = "/content/mkvmerge_workspace"
os.makedirs(WORK_DIR, exist_ok=True)
os.chdir(WORK_DIR)

print("\nâœ… Setup complete!")
print(f"ðŸ“ Working directory: {WORK_DIR}")
print(f"ðŸ’¾ Google Drive mounted")

In [ ]:
#@title ðŸ“¥ **STEP 2: Download Files**

download_method = widgets.Dropdown(
    options=['Google Drive (gdown)', 'Direct URL (aria2c)', 'Manual Upload'],
    value='Google Drive (gdown)',
    description='Method:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='400px')
)

url_input = widgets.Textarea(
    value='',
    placeholder='Enter Google Drive file ID or direct URL (one per line for multiple files)',
    description='URL/ID:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='600px', height='100px')
)

download_btn = widgets.Button(
    description='ðŸ“¥ Download',
    button_style='primary',
    icon='download',
    layout=widgets.Layout(width='150px')
)

upload_btn = widgets.Button(
    description='ðŸ“¤ Upload Files',
    button_style='info',
    icon='upload',
    layout=widgets.Layout(width='150px')
)

output_download = widgets.Output()

def download_files(b):
    with output_download:
        clear_output()
        method = download_method.value
        urls = [u.strip() for u in url_input.value.split('\n') if u.strip()]
        
        if not urls:
            print("âŒ Please enter at least one URL or file ID")
            return
        
        for url in urls:
            try:
                if 'Google Drive' in method:
                    print(f"ðŸ“¥ Downloading from Google Drive: {url}")
                    if 'drive.google.com' in url:
                        gdown.download(url, fuzzy=True)
                    else:
                        gdown.download(f'https://drive.google.com/uc?id={url}', fuzzy=True)
                else:
                    print(f"ðŸ“¥ Downloading with aria2c: {url}")
                    !aria2c -x 16 -s 16 "{url}"
                print(f"âœ… Download complete\n")
            except Exception as e:
                print(f"âŒ Error: {str(e)}\n")
        
        print("\nðŸ“‚ Files in workspace:")
        !ls -lh

def upload_files(b):
    with output_download:
        clear_output()
        print("ðŸ“¤ Select files to upload...")
        uploaded = files.upload()
        if uploaded:
            print(f"\nâœ… Uploaded {len(uploaded)} file(s)")
            print("\nðŸ“‚ Files in workspace:")
            !ls -lh

download_btn.on_click(download_files)
upload_btn.on_click(upload_files)

display(download_method)
display(url_input)
display(widgets.HBox([download_btn, upload_btn]))
display(output_download)

In [ ]:
#@title ðŸ“‚ **STEP 3: Select Files & Analyze Tracks**

selected_files = {
    'video': None,
    'audio': [],
    'subtitle': []
}

file_tracks = {}

def get_files_in_dir(directory="."):
    files_list = []
    for item in Path(directory).iterdir():
        if item.is_file():
            files_list.append(str(item.name))
    return sorted(files_list)

def analyze_tracks(filename):
    """Analyze file tracks using mkvmerge"""
    try:
        result = subprocess.run(
            ['mkvmerge', '-J', filename],
            capture_output=True,
            text=True
        )
        if result.returncode == 0:
            return json.loads(result.stdout)
        else:
            result = subprocess.run(
                ['mediainfo', '--Output=JSON', filename],
                capture_output=True,
                text=True
            )
            return json.loads(result.stdout) if result.returncode == 0 else None
    except:
        return None

def format_track_info(track_data, source_file):
    """Format track information for display"""
    tracks_info = []
    
    if not track_data:
        return [f"ðŸ“„ {source_file} (Unable to analyze)"]
    
    if 'tracks' in track_data:
        for track in track_data['tracks']:
            track_type = track.get('type', 'unknown')
            codec = track.get('codec', 'unknown')
            track_id = track.get('id', '?')
            
            lang = track.get('properties', {}).get('language', 'und')
            track_name = track.get('properties', {}).get('track_name', '')
            
            if track_type == 'video':
                width = track.get('properties', {}).get('pixel_dimensions', ['?', '?'])[0]
                height = track.get('properties', {}).get('pixel_dimensions', ['?', '?'])[1]
                info = f"ðŸŽ¬ Track {track_id}: Video | {codec} | {width}x{height}"
            elif track_type == 'audio':
                channels = track.get('properties', {}).get('audio_channels', '?')
                info = f"ðŸ”Š Track {track_id}: Audio | {codec} | {channels}ch | {lang}"
                if track_name:
                    info += f" | {track_name}"
            elif track_type == 'subtitles':
                info = f"ðŸ’¬ Track {track_id}: Subtitle | {codec} | {lang}"
                if track_name:
                    info += f" | {track_name}"
            else:
                info = f"â“ Track {track_id}: {track_type} | {codec}"
            
            tracks_info.append(info)
    
    return tracks_info if tracks_info else [f"ðŸ“„ {source_file} (No tracks found)"]

refresh_btn = widgets.Button(
    description='ðŸ”„ Refresh',
    button_style='info',
    icon='refresh',
    layout=widgets.Layout(width='120px')
)

video_selector = widgets.Dropdown(
    options=get_files_in_dir(),
    description='Video:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='500px')
)

audio_selector = widgets.SelectMultiple(
    options=get_files_in_dir(),
    description='Audio:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='500px', height='100px')
)

subtitle_selector = widgets.SelectMultiple(
    options=get_files_in_dir(),
    description='Subtitles:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='500px', height='100px')
)

analyze_btn = widgets.Button(
    description='ðŸ” Analyze Tracks',
    button_style='success',
    icon='search',
    layout=widgets.Layout(width='200px')
)

output_selection = widgets.Output()

def refresh_files(b):
    files_list = get_files_in_dir()
    video_selector.options = files_list
    audio_selector.options = files_list
    subtitle_selector.options = files_list
    with output_selection:
        clear_output()
        print("âœ… File list refreshed")

def analyze_selection(b):
    with output_selection:
        clear_output()
        
        selected_files['video'] = video_selector.value
        selected_files['audio'] = list(audio_selector.value)
        selected_files['subtitle'] = list(subtitle_selector.value)
        
        if not selected_files['video']:
            print("âŒ Please select a video file")
            return
        
        print("ðŸ” Analyzing selected files...\n")
        print("="*70)
        
        # Analyze video
        print(f"\nðŸ“¹ VIDEO FILE: {selected_files['video']}")
        print("-"*70)
        video_data = analyze_tracks(selected_files['video'])
        file_tracks[selected_files['video']] = video_data
        for track in format_track_info(video_data, selected_files['video']):
            print(track)
        
        # Analyze audio
        if selected_files['audio']:
            print(f"\nðŸ”Š AUDIO FILES: ({len(selected_files['audio'])})")
            print("-"*70)
            for audio_file in selected_files['audio']:
                print(f"\nðŸ“„ {audio_file}:")
                audio_data = analyze_tracks(audio_file)
                file_tracks[audio_file] = audio_data
                for track in format_track_info(audio_data, audio_file):
                    print(f"  {track}")
        
        # Analyze subtitles
        if selected_files['subtitle']:
            print(f"\nðŸ’¬ SUBTITLE FILES: ({len(selected_files['subtitle'])})")
            print("-"*70)
            for sub_file in selected_files['subtitle']:
                print(f"\nðŸ“„ {sub_file}:")
                sub_data = analyze_tracks(sub_file)
                file_tracks[sub_file] = sub_data
                for track in format_track_info(sub_data, sub_file):
                    print(f"  {track}")
        
        print("\n" + "="*70)
        print("âœ… Analysis complete! Proceed to STEP 4 for output preview.")

refresh_btn.on_click(refresh_files)
analyze_btn.on_click(analyze_selection)

display(refresh_btn)
display(HTML("<h4>Select Video File:</h4>"))
display(video_selector)
display(HTML("<h4>Select Audio Track(s) (Hold Ctrl/Cmd for multiple):</h4>"))
display(audio_selector)
display(HTML("<h4>Select Subtitle Track(s) (Hold Ctrl/Cmd for multiple):</h4>"))
display(subtitle_selector)
display(HTML("<br>"))
display(analyze_btn)
display(output_selection)

In [ ]:
#@title âš™ï¸ **STEP 4: Preview Output Structure**

track_config = widgets.Textarea(
    value='',
    placeholder='Optional: Add custom mkvmerge arguments\nExample: --language 0:eng --track-name "0:English Audio"',
    description='Custom Args:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='600px', height='80px')
)

output_name = widgets.Text(
    value='output_muxed.mkv',
    description='Output Name:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='400px')
)

gdrive_path = widgets.Text(
    value='/content/drive/MyDrive/',
    description='GDrive Path:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='500px')
)

copy_to_gdrive = widgets.Checkbox(
    value=True,
    description='Copy output to Google Drive',
    style={'description_width': 'initial'}
)

preview_btn = widgets.Button(
    description='ðŸ‘ï¸ Preview Output',
    button_style='info',
    icon='eye',
    layout=widgets.Layout(width='200px')
)

output_preview = widgets.Output()

def preview_output(b):
    with output_preview:
        clear_output()
        
        if not selected_files['video']:
            print("âŒ No video selected. Please go back to STEP 3 and analyze files.")
            return
        
        print("ðŸ‘ï¸ OUTPUT PREVIEW")
        print("="*70)
        print(f"\nðŸ“¦ Output File: {output_name.value}")
        print(f"ðŸ’¾ Will be saved to: {WORK_DIR}")
        if copy_to_gdrive.value:
            print(f"â˜ï¸ Will be copied to: {gdrive_path.value}")
        
        print("\nðŸ“‹ FINAL TRACK STRUCTURE:")
        print("="*70)
        
        track_counter = 0
        
        # Video tracks
        print(f"\nðŸ“¹ FROM: {selected_files['video']}")
        if selected_files['video'] in file_tracks:
            video_data = file_tracks[selected_files['video']]
            if video_data and 'tracks' in video_data:
                for track in video_data['tracks']:
                    if track.get('type') == 'video':
                        codec = track.get('codec', 'unknown')
                        width = track.get('properties', {}).get('pixel_dimensions', ['?', '?'])[0]
                        height = track.get('properties', {}).get('pixel_dimensions', ['?', '?'])[1]
                        print(f"  â†’ Track {track_counter}: Video | {codec} | {width}x{height}")
                        track_counter += 1
        
        # Audio tracks
        if selected_files['audio']:
            print(f"\nðŸ”Š AUDIO TRACKS: ({len(selected_files['audio'])})")
            for audio_file in selected_files['audio']:
                print(f"\nðŸ“„ FROM: {audio_file}")
                if audio_file in file_tracks:
                    audio_data = file_tracks[audio_file]
                    if audio_data and 'tracks' in audio_data:
                        for track in audio_data['tracks']:
                            if track.get('type') == 'audio':
                                codec = track.get('codec', 'unknown')
                                channels = track.get('properties', {}).get('audio_channels', '?')
                                lang = track.get('properties', {}).get('language', 'und')
                                track_name = track.get('properties', {}).get('track_name', '')
                                info = f"  â†’ Track {track_counter}: Audio | {codec} | {channels}ch | {lang}"
                                if track_name:
                                    info += f" | {track_name}"
                                print(info)
                                track_counter += 1
        
        # Subtitle tracks
        if selected_files['subtitle']:
            print(f"\nðŸ’¬ SUBTITLE TRACKS: ({len(selected_files['subtitle'])})")
            for sub_file in selected_files['subtitle']:
                print(f"\nðŸ“„ FROM: {sub_file}")
                if sub_file in file_tracks:
                    sub_data = file_tracks[sub_file]
                    if sub_data and 'tracks' in sub_data:
                        for track in sub_data['tracks']:
                            if track.get('type') == 'subtitles':
                                codec = track.get('codec', 'unknown')
                                lang = track.get('properties', {}).get('language', 'und')
                                track_name = track.get('properties', {}).get('track_name', '')
                                info = f"  â†’ Track {track_counter}: Subtitle | {codec} | {lang}"
                                if track_name:
                                    info += f" | {track_name}"
                                print(info)
                                track_counter += 1
        
        print("\n" + "="*70)
        print(f"ðŸ“Š Total tracks in output: {track_counter}")
        print("\nâœ… Preview complete! If everything looks good, proceed to STEP 5.")

preview_btn.on_click(preview_output)

display(HTML("<h4>Output Configuration:</h4>"))
display(output_name)
display(HTML("<h4>Custom Arguments (Optional):</h4>"))
display(track_config)
display(HTML("<h4>Google Drive Settings:</h4>"))
display(copy_to_gdrive)
display(gdrive_path)
display(HTML("<br>"))
display(preview_btn)
display(output_preview)

In [ ]:
#@title ðŸŽ¬ **STEP 5: START MUXING**

mux_btn = widgets.Button(
    description='ðŸŽ¬ START MUXING',
    button_style='success',
    icon='play',
    layout=widgets.Layout(width='250px', height='50px')
)

output_mux = widgets.Output()

def start_muxing(b):
    with output_mux:
        clear_output()
        
        if not selected_files['video']:
            print("âŒ Error: No video file selected!")
            print("Please go back to STEP 3 and select files.")
            return
        
        if not selected_files['audio'] and not selected_files['subtitle']:
            print("âŒ Error: Please select at least one audio or subtitle track!")
            return
        
        output_file = output_name.value
        
        print("="*70)
        print("ðŸŽ¬ STARTING MUXING PROCESS")
        print("="*70)
        print(f"\nðŸ“¹ Video: {selected_files['video']}")
        print(f"ðŸ”Š Audio tracks: {len(selected_files['audio'])}")
        print(f"ðŸ’¬ Subtitle tracks: {len(selected_files['subtitle'])}")
        print(f"ðŸ’¾ Output: {output_file}")
        print("\n" + "="*70)
        
        cmd_parts = ['mkvmerge', '-o', f'"{output_file}"']
        
        if track_config.value.strip():
            cmd_parts.append(track_config.value.strip())
        
        cmd_parts.append(f'"{selected_files["video"]}"')
        
        for audio in selected_files['audio']:
            cmd_parts.append(f'"{audio}"')
        
        for subtitle in selected_files['subtitle']:
            cmd_parts.append(f'"{subtitle}"')
        
        cmd = ' '.join(cmd_parts)
        
        print(f"\nðŸ”§ Executing mkvmerge...\n")
        
        result = os.system(cmd)
        
        if result == 0:
            print("\n" + "="*70)
            print("âœ… MUXING SUCCESSFUL!")
            print("="*70)
            print(f"\nðŸ“Š Output file info:")
            !ls -lh "{output_file}"
            
            if copy_to_gdrive.value:
                print(f"\nðŸ“¤ Copying to Google Drive...")
                gdrive_dest = os.path.join(gdrive_path.value, output_file)
                try:
                    shutil.copy2(output_file, gdrive_dest)
                    print(f"âœ… Copied to: {gdrive_dest}")
                except Exception as e:
                    print(f"âŒ Error copying to GDrive: {str(e)}")
            
            print("\nðŸŽ‰ All done!")
        else:
            print("\n" + "="*70)
            print("âŒ MUXING FAILED!")
            print("="*70)
            print("\nPlease check:")
            print("  â€¢ All selected files exist")
            print("  â€¢ File formats are supported")
            print("  â€¢ Custom arguments are valid")

mux_btn.on_click(start_muxing)

display(mux_btn)
display(output_mux)

In [ ]:
#@title 💾 **STEP 6: Download Output (Optional)**

download_output_btn = widgets.Button(
    description='⬇️ Download Output',
    button_style='warning',
    icon='download',
    layout=widgets.Layout(width='200px', height='40px')
)

output_download_section = widgets.Output()

def download_output(b):
    with output_download_section:
        clear_output()
        output_file = output_name.value
        if os.path.exists(output_file):
            print(f"📥 Downloading {output_file}...")
            print("⏳ This may take a while for large files...")
            files.download(output_file)
            print("✅ Download initiated!")
        else:
            print(f"❌ File '{output_file}' not found!")
            print("Please mux the files first.")

download_output_btn.on_click(download_output)

display(HTML("<p><i>Note: If you enabled 'Copy to Google Drive', the file is already in your Drive. Use this only if you need to download directly to your computer.</i></p>"))
display(download_output_btn)
display(output_download_section)